In [20]:
# imports
import subprocess
import numpy as np
import zipfile
import pandas as pd
import os
import gzip
import sys
import shutil
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from multiprocessing import Pool

In [21]:
# if youre on a mac and want to run the notebook without letting youre compouter fall asleep, remember to terminate
caffein = subprocess.Popen("caffeinate", shell=True)

## 0. Load  / (remove) data

In [22]:
# config
path = '/Users/max/code/maxwrf/expedia-ml' # os.getcwd()  # path to project directory
csvs = 'train.csv', 'test.csv', 'destinations.csv'  #  our names for the different csvs
gzs = '2013-2014.gz', '2015.gz', 'destinations.csv.gz'  # names of the gz files which we be unpacked from expedia.zip
url = "https://www.dropbox.com/s/gwrc8o99xcj6r72/Expedia.zip?dl=1" # dl=1
using_test = True
is_development = True
data_path = path + '/data/'
clear_hard_drive = True

In [23]:
# loads the dataset files into a passed directory otherwise current location
def load_files(url, gzs, csvs, data_path, path=os.getcwd()):
    os.chdir(path)

    if all(os.path.isfile(data_path + csv) for csv in csvs):
        print('Data already loaded')
    else:
        # if the Expedia file is not there yet init download it and write it as zip
        if not os.path.isfile('Expedia.zip'):
            print('Downloading Expedia.zip from Dropbox...')
            u = urllib.request.urlopen(url)
            data = u.read()
            u.close()
            
            total = 0
            with open('Expedia.zip', 'wb') as f:
                f.write(data)
                total += 1
                if total % 10000000 == 0:
                    print('Read {} lines...'.format(total))
            print('Finished downloading Expedia.zip from Dropbox')

        # Extract gz files from zip
        with zipfile.ZipFile("Expedia.zip", 'r') as zip_ref:
            print('Unzipping Expedia.zip')
            zip_ref.extractall(path)
            print('Finished unzipping Expedia.zip...')

        os.chdir(path + '/all')

        # extract the csvs from the gzs
        for csv, gz in zip(csvs, gzs):   
            with gzip.open(gz, 'rb') as f_in: 
                with open(csv, 'wb') as f_out:
                    print(f'writing {csv}...')
                    shutil.copyfileobj(f_in, f_out)
                    print(f'Finished writing {csv}')

        # clean up directory
        for f in gzs:
            os.remove(f)
            
        os.chdir(path)  # return to parent directory
        if os.path.exists(data_path): # if the data directory already exists probably empty, remove it
            shutil.rmtree(data_path)
        os.rename(path + '/all', data_path)
        os.remove('Expedia.zip')
        
        print(f'Finished.')

In [24]:
# laoding data into memory
def load_data(is_development, data_path, csvs, write_sample=False):
    print('Loading data into memory / dataframe')
    if not is_development:
        df_train = pd.read_csv(data_path + csvs[0])
        df_test = pd.read_csv(data_path + csvs[1])
        df_destination = pd.read_csv(data_path + csvs[2])  # not even needed. its enough to have the destination id as feature
    else:
        df_train = pd.read_csv(data_path + csvs[0], nrows=100000)
        df_test = pd.read_csv(data_path + csvs[1], nrows=10000)
        df_destination = pd.read_csv(data_path + csvs[2])
        
        if write_sample:
            print('Writing sample files...')
            df_train.to_csv(data_path + 'sample' + csvs[0], index=False)
            df_test.to_csv(data_path + 'sample' + csvs[1], index=False)
            print('Finished writing sample files...')
            
    print('Finished loading data into memory / dataframe')
    return df_train, df_test, df_destination

In [25]:
def remove_data(data_path, csvs, clear_hard_drive=False):
    if not clear_hard_drive:
        return
    print('Removing files from hard drive...')
    cwd = os.getcwd()
    os.chdir(data_path)
    for csv in csvs:
        try:
            print('Removing ', csv, '...')
            os.remove(csv)  
        except EnvironmentError:
            print(csv, ' not found.')
        try: 
            print('Removing sample ', csv, '...')
            os.remove('sample' + csv)
        except EnvironmentError:
            print('sample', csv, ' not found.')
    os.chdir(cwd)
    print('Finished removing files from hard drive...')

In [26]:
load_files(url, gzs, csvs, data_path, path)

Data already loaded


In [27]:
df_train, df_test, df_destination = load_data(is_development, data_path, csvs, write_sample=True)

Loading data into memory / dataframe
Writing sample files...
Finished writing sample files...
Finished loading data into memory / dataframe


In [28]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
date_time                    100000 non-null object
site_name                    100000 non-null int64
posa_continent               100000 non-null int64
user_location_country        100000 non-null int64
user_location_region         100000 non-null int64
user_location_city           100000 non-null int64
orig_destination_distance    63078 non-null float64
user_id                      100000 non-null int64
is_mobile                    100000 non-null int64
is_package                   100000 non-null int64
channel                      100000 non-null int64
srch_ci                      99929 non-null object
srch_co                      99929 non-null object
srch_adults_cnt              100000 non-null int64
srch_children_cnt            100000 non-null int64
srch_rm_cnt                  100000 non-null int64
srch_destination_id          100000 non-null int64
srch_destination_type

## 1. What is the challenge?
Expedia is interested in predicting which hotel group a user is going to book.
* Does that mean in test set every row we look at eventually is a booking and the feature is the hotel class (multi class problem)
* Hence, easiest approach, drop all rows on other 'clicking events' and simply consider rows where is_booking
* But beforehadn construct feature over all rows
* Could you alternatively do, is the even going to be a booking? 1 / 0

## 2. EDA

In [29]:
# @Pier, maybe you could use these copies of the DFs in the EDA such that if we proceed to the feature engineering,
# we know what the dfs are still in the raw loaded format
df_train_eda = df_train.copy(deep=True)
df_test_eda = df_test.copy(deep=True)

## 3. Clean the data / engineer features

In [30]:
"""Function to check which columns contain NANS"""
def checkna(df):
    for col in df.columns:
        if df[col].isnull().any():
            print(col, '| Nan:', (df[col].isnull()).sum())

In [31]:
"""Function to fill NANS and change time stamp foremats"""
def fillna_convert(df):
    # filling empty orig_destianation distance
    df['orig_destination_distance'].fillna(df['orig_destination_distance'].median(), inplace=True)
    
    # make timestemps pd datetimes
    for col in df[['date_time', 'srch_ci', 'srch_co']].columns:
        df[col] = pd.to_datetime(df[col])
        
    return df

In [32]:
"""A decorater to keep track of the function calls to the wrapped function"""
# TODO: Does not account for multi threading yet
def counted(func):
    def wrapped(*args, **kwargs):
        wrapped.calls += 1
        if wrapped.calls % 10000 == 0:
            print(f'{func.__name__}: {wrapped.calls} rows...')
        return func(*args, **kwargs)
    wrapped.calls = 0
    return wrapped

In [33]:
"""Function to engineer features on the given df, aggregates user info from df_full"""
def engineer_features(df, df_full):
    # length of trip
    df['trip_length'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['trip_length'].fillna((df['trip_length'].median()), inplace=True)
    
    # is the trip in the same country
    df['domestic'] = np.where((df['user_location_country'].equals(df['hotel_country'])), 1, 0)
    
    # is it trip length smaller than 3 days
    df['short_trip'] = np.where((df['trip_length'] <= 3), 1, 0)
    
    #  is it a weekend trip
    df['srch_ci_d'] = df['srch_ci'].dt.day_name()
    df['srch_co_d'] = df['srch_co'].dt.day_name()
    df['weekend_trip'] = np.where((((df['srch_ci_d']=='Friday') & 
                                   (df['trip_length'] <= 3)) | 
                                  ((df['srch_ci_d']=='Saturday') &
                                   (df['trip_length'] <= 2))), 1, 0)
    
    # is it a business trip
    df['business_trip'] = np.where(((df['srch_ci_d'] != 'Friday') &
                                    (df['srch_ci_d'] != 'Saturday') &
                                    (df['srch_ci_d'] != 'Sunday') &
                                    (df['srch_co_d'] != 'Saturday') &
                                    (df['srch_co_d'] != 'Sunday') & 
                                    (df['trip_length'] <= 4)), 1, 0)
    df.drop(columns=['srch_ci_d', 'srch_co_d'], inplace=True)
    
    # plan time - how far ahead do we plan the trip
    df['plan_time'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    # is it a solo trip / family trip
    df['solo_trip'] = np.where(((df['srch_adults_cnt'] == 1) & 
                                (df['srch_children_cnt'] == 0)), 1, 0)
    
    # aggregate a mean booking rate
    @counted
    def aggregated_booking_rate(instance):
        if instance['is_booking'] == 0:
            return np.nan
        instance_date = instance['date_time']
        instance_id = instance['user_id']
        mean_booking_rate = df_full[(df_full['date_time'] <= instance_date) & 
                                    (df_full['user_id'] == instance_id)]['is_booking'].mean()
        return mean_booking_rate
        
    df['booking_rate'] = df.apply(aggregated_booking_rate, axis=1)
    
    # aggregate previous bookings & clicks by hotel cluster
    @counted
    def aggregated_previous_cluster(instance, hotel_cluster):
        if instance['is_booking'] == 0:
            return np.nan, np.nan
        instance_date = instance['date_time']
        instance_id = instance['user_id']
        count_booked = len(df_full[(df_full['date_time'] <= instance_date) & 
                                   (df_full['user_id'] == instance_id) & 
                                   (df_full['hotel_cluster'] == hotel_cluster) & 
                                   (df_full['is_booking'] == 1)])
        count_not_booked = len(df_full[(df_full['date_time'] <= instance_date) & 
                                       (df_full['user_id'] == instance_id) & 
                                       (df_full['hotel_cluster'] == hotel_cluster) & 
                                       (df_full['is_booking'] == 0)])
        return count_booked, count_not_booked
            
    for hotel_cluster in df_full['hotel_cluster'].unique():
        if np.isnan(hotel_cluster):  # test set does not have cluster given
            continue
        df['booked_cluster' + str(int(hotel_cluster))], df['not_booked_cluster' + str(int(hotel_cluster))]  = zip(*df.apply(lambda instance: aggregated_previous_cluster(instance, hotel_cluster), axis=1))

    return df

"""Function to enable multi threading for feature engineering"""
def parallel_feature_engineering(df_to_split, df_full, func=engineer_features, n_cores=4):
    df_splits = np.array_split(df_to_split, n_cores)
    args = [[df_split, df_full] for df_split in df_splits]
    pool = Pool(n_cores)
    df = pd.concat(pool.starmap(func, args))
    pool.close()
    pool.join()
    return df

In [34]:
"""Function removing features not needed for training"""
def finalize(df):
    for col in ['date_time', 'srch_ci', 'srch_co', 'user_id']:
        try:
            df.drop(col, axis=1, inplace=True)
        except KeyError:
            pass
    return df

In [35]:
"""Function to build features for train df"""
def prepare_df_train(df_train):
    df_train = fillna_convert(df_train)
    df_train = parallel_feature_engineering(df_train, df_train)
    df_train = finalize(df_train)
    
    # current strategy is to only use rows where booking is one and aggregate all info
    len_before = len(df_train)
    df_train = df_train[df_train['is_booking'] == 1]
    df_train = df_train.drop('is_booking', axis=1)
    print(f'Dropped {len_before - len(df_train)} rows which did not represent a booking in df train.')
    
    return df_train

In [36]:
"""Function to build features for train df"""
def prepare_df_test(df_test, df_train):
    df_test = fillna_convert(df_test)
    df_train = fillna_convert(df_train)
    
    # to engineer features on the test set we will concate test and train and aggregate user info from train on test
    df_train['is_Train'] = True
    df_test['is_Train'] = False
    df_test['is_booking'] = 1
    df_test['hotel_cluster'] = np.nan
    df_test_full_history = df_train.append(df_test)
    df_test = parallel_feature_engineering(df_test, df_test_full_history)
    df_test = df_test.drop(['is_booking', 'is_Train', 'hotel_cluster', 'id'], axis=1)
    df_test = finalize(df_test)
    return df_test

In [37]:
def save_prepared(df, data_path, path):
    os.chdir(data_path)
    df.to_csv(data_path + df.__name__)
    os.chdir(path)

In [38]:
df_train_prepared = prepare_df_train(df_train)

if using_test:
    # there is also the 'cnt' feature in train which is not in test and threfore needs to be dropped
    df_train_prepared.drop(['cnt'], axis=1, inplace=True)
    df_train.drop(['cnt'], axis=1, inplace=True)
    df_test_prepared = prepare_df_test(df_test, df_train)

aggregated_booking_rate: 10000 rows...
aggregated_booking_rate: 10000 rows...
aggregated_booking_rate: 10000 rows...
aggregated_booking_rate: 10000 rows...
aggregated_booking_rate: 20000 rows...
aggregated_booking_rate: 20000 rows...
aggregated_booking_rate: 20000 rows...
aggregated_booking_rate: 20000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...


aggregated_previous_cluster: 460000 rows...
aggregated_previous_cluster: 460000 rows...
aggregated_previous_cluster: 460000 rows...
aggregated_previous_cluster: 470000 rows...
aggregated_previous_cluster: 470000 rows...
aggregated_previous_cluster: 460000 rows...
aggregated_previous_cluster: 470000 rows...
aggregated_previous_cluster: 470000 rows...
aggregated_previous_cluster: 480000 rows...
aggregated_previous_cluster: 480000 rows...
aggregated_previous_cluster: 480000 rows...
aggregated_previous_cluster: 490000 rows...
aggregated_previous_cluster: 490000 rows...
aggregated_previous_cluster: 480000 rows...
aggregated_previous_cluster: 490000 rows...
aggregated_previous_cluster: 500000 rows...
aggregated_previous_cluster: 500000 rows...
aggregated_previous_cluster: 490000 rows...
aggregated_previous_cluster: 500000 rows...
aggregated_previous_cluster: 510000 rows...
aggregated_previous_cluster: 510000 rows...
aggregated_previous_cluster: 500000 rows...
aggregated_previous_cluster: 510

aggregated_previous_cluster: 910000 rows...
aggregated_previous_cluster: 930000 rows...
aggregated_previous_cluster: 940000 rows...
aggregated_previous_cluster: 920000 rows...
aggregated_previous_cluster: 930000 rows...
aggregated_previous_cluster: 940000 rows...
aggregated_previous_cluster: 950000 rows...
aggregated_previous_cluster: 940000 rows...
aggregated_previous_cluster: 930000 rows...
aggregated_previous_cluster: 950000 rows...
aggregated_previous_cluster: 960000 rows...
aggregated_previous_cluster: 950000 rows...
aggregated_previous_cluster: 940000 rows...
aggregated_previous_cluster: 960000 rows...
aggregated_previous_cluster: 970000 rows...
aggregated_previous_cluster: 950000 rows...
aggregated_previous_cluster: 970000 rows...
aggregated_previous_cluster: 960000 rows...
aggregated_previous_cluster: 980000 rows...
aggregated_previous_cluster: 970000 rows...
aggregated_previous_cluster: 960000 rows...
aggregated_previous_cluster: 980000 rows...
aggregated_previous_cluster: 990

aggregated_previous_cluster: 1390000 rows...
aggregated_previous_cluster: 1380000 rows...
aggregated_previous_cluster: 1400000 rows...
aggregated_previous_cluster: 1370000 rows...
aggregated_previous_cluster: 1400000 rows...
aggregated_previous_cluster: 1390000 rows...
aggregated_previous_cluster: 1410000 rows...
aggregated_previous_cluster: 1380000 rows...
aggregated_previous_cluster: 1410000 rows...
aggregated_previous_cluster: 1400000 rows...
aggregated_previous_cluster: 1420000 rows...
aggregated_previous_cluster: 1390000 rows...
aggregated_previous_cluster: 1420000 rows...
aggregated_previous_cluster: 1410000 rows...
aggregated_previous_cluster: 1400000 rows...
aggregated_previous_cluster: 1430000 rows...
aggregated_previous_cluster: 1430000 rows...
aggregated_previous_cluster: 1420000 rows...
aggregated_previous_cluster: 1440000 rows...
aggregated_previous_cluster: 1410000 rows...
aggregated_previous_cluster: 1440000 rows...
aggregated_previous_cluster: 1430000 rows...
aggregated

aggregated_previous_cluster: 1850000 rows...
aggregated_previous_cluster: 1820000 rows...
aggregated_previous_cluster: 1860000 rows...
aggregated_previous_cluster: 1840000 rows...
aggregated_previous_cluster: 1860000 rows...
aggregated_previous_cluster: 1870000 rows...
aggregated_previous_cluster: 1830000 rows...
aggregated_previous_cluster: 1850000 rows...
aggregated_previous_cluster: 1870000 rows...
aggregated_previous_cluster: 1880000 rows...
aggregated_previous_cluster: 1840000 rows...
aggregated_previous_cluster: 1860000 rows...
aggregated_previous_cluster: 1880000 rows...
aggregated_previous_cluster: 1890000 rows...
aggregated_previous_cluster: 1850000 rows...
aggregated_previous_cluster: 1870000 rows...
aggregated_previous_cluster: 1890000 rows...
aggregated_previous_cluster: 1900000 rows...
aggregated_previous_cluster: 1860000 rows...
aggregated_previous_cluster: 1880000 rows...
aggregated_previous_cluster: 1900000 rows...
aggregated_previous_cluster: 1910000 rows...
aggregated

aggregated_previous_cluster: 2290000 rows...
aggregated_previous_cluster: 2270000 rows...
aggregated_previous_cluster: 2320000 rows...
aggregated_previous_cluster: 2300000 rows...
aggregated_previous_cluster: 2320000 rows...
aggregated_previous_cluster: 2280000 rows...
aggregated_previous_cluster: 2330000 rows...
aggregated_previous_cluster: 2330000 rows...
aggregated_previous_cluster: 2310000 rows...
aggregated_previous_cluster: 2340000 rows...
aggregated_previous_cluster: 2290000 rows...
aggregated_previous_cluster: 2340000 rows...
aggregated_previous_cluster: 2320000 rows...
aggregated_previous_cluster: 2350000 rows...
aggregated_previous_cluster: 2300000 rows...
aggregated_previous_cluster: 2350000 rows...
aggregated_previous_cluster: 2330000 rows...
aggregated_previous_cluster: 2360000 rows...
aggregated_previous_cluster: 2310000 rows...
aggregated_previous_cluster: 2360000 rows...
aggregated_previous_cluster: 2340000 rows...
aggregated_previous_cluster: 2370000 rows...
aggregated

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 10000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 20000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 30000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 40000 rows...
aggregated_previous_cluster: 50000 rows...
aggregated_previous_cluster: 50000 rows...
aggregated_previous_cluster: 50000 rows...
aggregated_previous_cluster: 50000 rows...
aggregated_previous_cluster: 60000 rows...
aggregated_previous_cluster: 60000 rows...
aggregated_previous_cluster: 60000 rows...
aggregated_

In [39]:
pd.set_option('display.max_columns', None)
df_train_prepared.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,hotel_cluster,trip_length,domestic,short_trip,weekend_trip,business_trip,plan_time,solo_trip,booking_rate,booked_cluster1,not_booked_cluster1,booked_cluster80,not_booked_cluster80,booked_cluster21,not_booked_cluster21,booked_cluster92,not_booked_cluster92,booked_cluster41,not_booked_cluster41,booked_cluster69,not_booked_cluster69,booked_cluster70,not_booked_cluster70,booked_cluster98,not_booked_cluster98,booked_cluster10,not_booked_cluster10,booked_cluster18,not_booked_cluster18,booked_cluster28,not_booked_cluster28,booked_cluster25,not_booked_cluster25,booked_cluster2,not_booked_cluster2,booked_cluster16,not_booked_cluster16,booked_cluster94,not_booked_cluster94,booked_cluster77,not_booked_cluster77,booked_cluster24,not_booked_cluster24,booked_cluster58,not_booked_cluster58,booked_cluster36,not_booked_cluster36,booked_cluster82,not_booked_cluster82,booked_cluster46,not_booked_cluster46,booked_cluster30,not_booked_cluster30,booked_cluster29,not_booked_cluster29,booked_cluster57,not_booked_cluster57,booked_cluster85,not_booked_cluster85,booked_cluster5,not_booked_cluster5,booked_cluster59,not_booked_cluster59,booked_cluster62,not_booked_cluster62,booked_cluster81,not_booked_cluster81,booked_cluster6,not_booked_cluster6,booked_cluster53,not_booked_cluster53,booked_cluster8,not_booked_cluster8,booked_cluster73,not_booked_cluster73,booked_cluster26,not_booked_cluster26,booked_cluster95,not_booked_cluster95,booked_cluster13,not_booked_cluster13,booked_cluster42,not_booked_cluster42,booked_cluster9,not_booked_cluster9,booked_cluster55,not_booked_cluster55,booked_cluster91,not_booked_cluster91,booked_cluster72,not_booked_cluster72,booked_cluster65,not_booked_cluster65,booked_cluster56,not_booked_cluster56,booked_cluster68,not_booked_cluster68,booked_cluster75,not_booked_cluster75,booked_cluster38,not_booked_cluster38,booked_cluster67,not_booked_cluster67,booked_cluster78,not_booked_cluster78,booked_cluster43,not_booked_cluster43,booked_cluster37,not_booked_cluster37,booked_cluster99,not_booked_cluster99,booked_cluster35,not_booked_cluster35,booked_cluster20,not_booked_cluster20,booked_cluster90,not_booked_cluster90,booked_cluster50,not_booked_cluster50,booked_cluster22,not_booked_cluster22,booked_cluster64,not_booked_cluster64,booked_cluster60,not_booked_cluster60,booked_cluster11,not_booked_cluster11,booked_cluster97,not_booked_cluster97,booked_cluster89,not_booked_cluster89,booked_cluster83,not_booked_cluster83,booked_cluster14,not_booked_cluster14,booked_cluster51,not_booked_cluster51,booked_cluster15,not_booked_cluster15,booked_cluster40,not_booked_cluster40,booked_cluster45,not_booked_cluster45,booked_cluster84,not_booked_cluster84,booked_cluster54,not_booked_cluster54,booked_cluster76,not_booked_cluster76,booked_cluster17,not_booked_cluster17,booked_cluster49,not_booked_cluster49,booked_cluster32,not_booked_cluster32,booked_cluster7,not_booked_cluster7,booked_cluster88,not_booked_cluster88,booked_cluster79,not_booked_cluster79,booked_cluster4,not_booked_cluster4,booked_cluster39,not_booked_cluster39,booked_cluster47,not_booked_cluster47,booked_cluster19,not_booked_cluster19,booked_cluster33,not_booked_cluster33,booked_cluster48,not_booked_cluster48,booked_cluster93,not_booked_cluster93,booked_cluster0,not_booked_cluster0,booked_cluster34,not_booked_cluster34,booked_cluster63,not_booked_cluster63,booked_cluster96,not_booked_cluster96,booked_cluster44,not_booked_cluster44,booked_cluster3,not_booked_cluster3,booked_cluster12,not_booked_cluster12,booked_cluster86,not_booked_cluster86,booked_cluster71,not_booked_cluster71,booked_cluster61,not_booked_cluster61,booked_cluster66,not_booked_cluster66,booked_cluster52,not_booked_cluster52,booked_cluster87,not_booked

In [40]:
# should only show the hotel cluster
set(df_train_prepared.columns.to_list() + df_test_prepared.columns.to_list()) - set(df_test_prepared.columns.to_list())

{'hotel_cluster'}

In [41]:
print(os.getcwd())
df_train_prepared.to_csv('prepared_train.csv')
df_test_prepared.to_csv('prepared_test.csv')
df_train = df_train_prepared
df_test = df_test_prepared

/Users/max/code/maxwrf/expedia-ml


## 4. Models

In [42]:
"""
TODO: 
We are generating features on past customer data, so we have more infor available for instances with later
timestamps.

If we use the train data, which is unlabled, we aggregate features for its users over the full train set. Thus,
we do not address more recent click events.

If we only use the test data, and split it into test and train, we would have labeled test data with recent click
events (More of a real time situation?). A problem which would still prevail that for the early instances we do not
have any previous click events or bookings - How does that impact our training?
"""

X_train = df_train.drop(['hotel_cluster'], axis=1)
y_train = df_train['hotel_cluster']
X_test = df_test

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import warnings

warnings.simplefilter("ignore")

forest = RandomForestClassifier(n_jobs=-1)
forest.fit(X_train, y_train)

if using_test:
    forest_preds_test = forest.predict(X_test)

np.mean(cross_val_score(forest, X_train, y_train, cv=10, scoring='accuracy'))

0.4762998790810157

In [44]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

if using_test:
    tree_preds_test = tree.predict(X_test)

np.mean(cross_val_score(tree, X_train, y_train, cv=10, scoring='accuracy'))

0.40991535671100365

In [45]:
# feature importance in random forest
feature_importances = pd.DataFrame(forest.feature_importances_, index = X_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
srch_destination_id,0.027391
hotel_market,0.026706
booking_rate,0.026435
plan_time,0.023853
orig_destination_distance,0.023112
...,...
not_booked_cluster74,0.000372
booked_cluster92,0.000323
booked_cluster74,0.000282
booked_cluster27,0.000215


In [46]:
# feature importance in decision tree
feature_importances = pd.DataFrame(tree.feature_importances_, index = X_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
hotel_market,0.042984
srch_destination_id,0.032479
plan_time,0.029491
orig_destination_distance,0.028777
booking_rate,0.028027
...,...
not_booked_cluster88,0.000293
not_booked_cluster31,0.000245
not_booked_cluster66,0.000220
not_booked_cluster27,0.000123


In [47]:
import xgboost as xgb
from xgboost import XGBClassifier

xgb = XGBClassifier(n_jobs=-1, max_depth=2, eta=1)

X_train_xgb = X_train.loc[:,feature_importances.index[:50]]
X_test_xgb = X_test.loc[:,feature_importances.index[:50]]
xgb.fit(X_train_xgb, y_train)

xgb_preds_test = xgb.predict(X_test_xgb)

np.mean(cross_val_score(xgb, X_train, y_train, cv=10, scoring='accuracy'))

0.5541717049576784

## 4. Remove data / kill subproccess

In [48]:
remove_data(data_path, csvs, clear_hard_drive)

Removing files from hard drive...
Removing  train.csv ...
Removing sample  train.csv ...
Removing  test.csv ...
Removing sample  test.csv ...
Removing  destinations.csv ...
Removing sample  destinations.csv ...
sample destinations.csv  not found.
Finished removing files from hard drive...


In [49]:
caffein.kill()